In [1]:
import numpy as np
from typing import List, Optional, Union, Dict
from scipy.special import logsumexp
import itertools
import os

# Bayesian Networks and EM

In the following questions we will be considering a Bayesian Network for modelling variables related to students' ability to get a job (note this is a toy example aiming to encode a few simple intuitions, the values are randomly made).

It has the following variables:
- Difficulty (D) of a job-related course in the year the student took the course
- Effort (E) that the student put into the course and then finding a job
- Aptitude (A) of the student
- Confidence (C) of the student in the related subject
- Grade (G) of the student in the course
- Interview (I) that the student took for the job
- TuteAttendance (T) of the student in the course
- ForumParticipation (F) of the student in the course
- SAT (S) that the student got (Scholarly Aptitute Test, a rough measure of Aptitude)
- Job (J) - whether the student got the job or not

These variables will be the nodes of our Bayesian Network. We will specify how we believe the variables are affected by each other by the structure of the graph, for example Grade's parents are Difficulty, Effort and Aptitude as the values for those variables directly affect the value of Grade.

To represent the Bayesian network, we first define two classes, one for Probability Mass Function objects (`PMF`) that have an array and what each axis means, and the second is a general Bayesian Network node (`BNNode`) that has the nodes' state values, its parents, and its pmf (an object on type `PMF`).

The PMF array is always meant to be a conditional probability table, with the probabilities given for the last column given the previous columns. For example `PMF(['Difficulty', 'Effort', 'Aptitude', 'Grade'], (3, 3, 5, 5))` represents $p(G|D,E,A)$, and $p(G=g|D=d,E=e,A=a)$ is at index `[d,e,a,g]` of the array.

You should read through the definition of these classes, as you will be working with these objects.

In [2]:
## Supporting Cell Do NOT change
class PMF:
    def __init__(self, dims: List['BNNode'], values: np.array):
        self.dims = dims
        self.values = values
    def __repr__(self):
        return "PMF({}, {})".format(vars2names(self.dims), self.values.shape)

class BNNode:
    def __init__(self, name, states: List, parents: [List['BNNode']], pmf):
        self.name = name
        self.states = states
        self.num_states = len(self.states)
        self.parents = parents
        self.pmf = pmf # {'dim_vars': [names], 'value':pmf} where pmf is (num_states,) or (parents1.num_states,...,num_states)
    def check_pmf(self, pmf):
        assert type(pmf) == PMF, "Expect pmf to be of type PMF, got {}".format(type(pmf))
        assert len(pmf.dims)>0, "Expect at least 1 dim, got {} dims".format(len(pmf.dims))
        assert self == pmf.dims[-1], "Last dim should be {}, got {}".format(self.name, pmf.dims[-1].name)
        # Make sure the pmf is of the correct shape
        expected_shapes = tuple(var.num_states for var in pmf.dims)
        assert pmf.values.shape == expected_shapes,  \
            "Got shape {} for pmf, expected {}".format(pmf.values.shape, expected_shapes)
        assert np.allclose(pmf.values.sum(axis=-1), 1), "Not all last dims sum to 1: sums: {}".format(pmf.values.sum(axis=-1))
    def __eq__(self, other):
        """Define equality of nodes if they have the same name"""
        if isinstance(other, BNNode):
            return self.name == other.name
        return False
    def __hash__(self):
        return hash(self.name)
    def __repr__(self):
        return "BNNode[{}]".format(self.name)
        
# Useful for debugging!
vars2names = lambda var_lst: [var.name for var in var_lst]

Next we will actually define a specific Bayesian Network for our problem instance using our BN node. Note this has defined the parents of each node, which defines the structue of the graph.

In [3]:
## Supporting Cell Do NOT change
D = BNNode("Difficulty",['Easy', 'Med', 'Hard'], [], None)
E = BNNode("Effort",['Low', 'Med', 'High'], [], None)
A = BNNode("Aptitute",[1,2,3,4,5], [], None)
C = BNNode("Confidence",['Low', 'Med', 'High'], [], None)
G = BNNode("Grade",['F', 'P', 'Cr', 'D', 'HD'], [D,E,A], None)
I = BNNode("Interview",['Bad', 'Ok', 'Godd'], [E,A,C], None)
T = BNNode("TuteAttendence",['Low', 'Med', 'High'], [E,C], None)
F = BNNode("ForumParticipation",['Low', 'Med', 'High'], [E,A,C], None)
S = BNNode("SAT",['Low', 'Med', 'High'], [A], None)
J = BNNode("Job",['No', 'Yes'], [G,I], None)

nodes = [D, E, A, C, G, I, T, F, S, J]
print(vars2names(nodes))

['Difficulty', 'Effort', 'Aptitute', 'Confidence', 'Grade', 'Interview', 'TuteAttendence', 'ForumParticipation', 'SAT', 'Job']


## BN Part 1: General Properties
### Question 1.1 [4 marks]

1. Draw the Bayesian Network structure for the network defined above (note the structure is defined by the parents of the node specified when declaring each node).
2. Write down the factorisation of the joint distribution given this structure.
3. Calculate the number of parameters needed to represent the total joint distribution with this factorisation and without any factorisation.

### Solution

1.1.1
![title](submission_extras/1.png)  
1.1.2
![title](submission_extras/2.png)
1.1.3
If we don't use the factorisation, we will need $\prod_{v \in V}{number of p(V=v)}= 3*3*5*3*5*3*3*3*3*2 = 109,350$ parameters.  
If we use the the factorisation, we only need $\sum_{v \in V}{number of p(V=v)} = 3+3+5+3+5+3+3+3+3+2=33$ parameters.


### Question 1.2 [6 marks]
Do the following conditional indpendencies hold? Prove your answer.

1. $C \perp \!\!\! \perp G\mid I$
2. $C \perp \!\!\! \perp G\mid E$
3. $C \perp \!\!\! \perp G\mid I,E$
4. $C \perp \!\!\! \perp G\mid I,E,J$
5. $F \perp \!\!\! \perp J\mid I,G$
6. $F \perp \!\!\! \perp J\mid I,E,A$



### Solution

1.  
We can give a counterexample (an unblocked path) for this. Observe that for this path: C-I-A-G. Firstly, C-I-A forms observed HH-node, which is unblocked. Secondly, I-A-G forms unobserved TT-node, which is also unblocked. Hence we find an unblocked path: C-I-A-G. Therefore, $C \perp \!\!\! \perp G\mid I$ does not hold. 


2.  
We will go through all six acyclic paths from C to G to prove that they are all blocked.  
(1) We find that C-I-A forms unobserved HH-node, which is blocked. Then path C-I-A-G is blocked.  
(2) C-F-A forms unobserved HH-node, which is blocked. Then path C-F-A-G is blocked.  
(3) T-E-G forms observed TT-node, which is blocked. Then path C-T-E-G is blocked.  
(4) I-J-G forms unobserved HH-node, which is blocked. Then path C-I-J-G is blocked.  
(5) F-E-G forms observed TT-node, which is blocked. Then path C-F-E-G is blocked.  
(6) I-E-G forms observed TT-node, which is blocked. Then path C-I-E-G is blocked.  
Therefore, $C \perp \!\!\! \perp G\mid E$ holds.


3.  
We can give a counterexample (an unblocked path) for this. Observe that for this path: C-I-A-G. Firstly, C-I-A forms observed HH-node, which is unblocked. Secondly, I-A-G forms unobserved TT-node, which is also unblocked. Hence we find an unblocked path: C-I-A-G. Therefore, $C \perp \!\!\! \perp G\mid I,E$ does not hold. 


4.  
We can give a counterexample (an unblocked path) for this. Observe that for this path: C-I-A-G. Firstly, C-I-A forms observed HH-node, which is unblocked. Secondly, I-A-G forms unobserved TT-node, which is also unblocked. Hence we find an unblocked path: C-I-A-G. Therefore, $C \perp \!\!\! \perp G\mid I,E,J$ does not hold. 


5.  
There are many paths from J to F, and we find that all of them will pass node I and G. Hence we only need to prove all J-G-Xs and J-I-Xs are blocked. We find that all Xs (including D, E, A, C) point to I and G, both I and G  point to J, and only I,G are observed in the graph. We can assert that all J-G-Xs and J-I-Xs are observed HT-nodes and hence all paths from J to F are blocked.
Therefore, $F \perp \!\!\! \perp J\mid I,G$ holds.


6.  
We will go through all six acyclic paths from F to J to prove that they are all blocked.  
(1) we find that C-I-J forms observed HT-node, which is blocked. Then path F-C-I-J is blocked.  
(2) A-I-J forms observed HT-node, which is blocked. Then path F-A-I-J is blocked.  
(3) E-I-J forms observed HT-node, which is blocked. Then path F-E-I-J is blocked.  
(4) F-A-G forms observed TT-node, which is blocked. Then path F-A-G-J is blocked.  
(5) F-E-G forms observed TT-node, which is blocked. Then path F-E-G-J is blocked.  
(6) T-E-G forms observed TT-node, which is blocked. Then path F-C-T-E-G-J is blocked.  
Therefore, $F \perp \!\!\! \perp J\mid I,E,A$ holds.

# BN Part 2: Learning the Parameters from Data

We now want to learn the parameters of the model given observed samples. However we will only have observed samples for some of the variables, specifically all the variables except `Aptitude`. Thus $A$ is a latent variable, and all other variables are observed. Furthermore we will have some parameters of the model, specifically $p(S\mid A)$ and $p(F\mid E,A,C)$. We can think of this as putting in information from other sources: we have found a model (maybe from previous research) for how SAT scores $S$ are affected by Aptitude and how Forum Participation $F$ are affected by Effort, Aptitude and Confidence, so we put this information in directly.

However as we have a latent variable, we won't be able to solve directly for all the other unknown parameters using Maximum Likelihood Estimation (MLE). So instead we will need to use Expectation Maximisation (EM) to be able to learn the parameters.

We first initialise the (other) pmfs to uniform probability, and load in observed samples. The format for the sample data, `sample_idxes`, is a $n\times d$ array where $n$ is the number of samples (5000) and $d$ is the number of nodes (10). The value in each column is the index of the observed state for that variable, and the columns are in the same order as `nodes`.

As `A` is not observed, the indices for that column are all -1.
Note that the observed values are encoded as 0:K-1, where K is the number of states in the node. 

In [4]:
## Supporting Cell Do NOT change

# Initialise parameters to uniform
D.pmf = PMF([D], np.ones((3,))/3)
E.pmf = PMF([E], np.ones((3,))/3)
A.pmf = PMF([A], np.ones((5,))/5)
C.pmf = PMF([C], np.ones((3,))/3)

G.pmf = PMF([D,E,A,G], np.ones((3,3,5,5))/(5))
I.pmf = PMF([E,A,C,I], np.ones((3,5,3,3))/(3))
T.pmf = PMF([E,C,T], np.ones((3,3,3))/(3))
F.pmf = PMF([E,A,C,F], np.ones((3,5,3,3))/(3))
S.pmf = PMF([A,S], np.ones((5,3))/(3))
J.pmf = PMF([G,I,J], np.ones((5,3,2))/(2))

# Load and set parameters for specific variables
S_pmf = np.load(os.path.join("data","question_1","S_pmf.npy"))
S.pmf.values = S_pmf
F_pmf = np.load(os.path.join("data","question_1","F_pmf.npy"))
F.pmf.values = F_pmf

for var in nodes:
    var.check_pmf(var.pmf) # Make sure the pmfs are valid

In [5]:
## Supporting Cell Do NOT change

# examine one of the loaded pmfs P(S|A)
# Aptitute x SAT
print(S.pmf.__repr__())
print(S.pmf.values)


# print(G.pmf)
# print(G.pmf.values)

PMF(['Aptitute', 'SAT'], (5, 3))
[[0.7  0.25 0.05]
 [0.6  0.3  0.1 ]
 [0.4  0.5  0.1 ]
 [0.2  0.6  0.2 ]
 [0.05 0.45 0.5 ]]


In [6]:
## Supporting Cell Do NOT change

# You're given a dataset consisting of records of 5000 students
# load this, and examine the first three rows
sample_idxes = np.load(os.path.join("data","question_1","sample_idxes_data.npy"))
print(sample_idxes.shape)
print(sample_idxes[0:3])

(5000, 10)
[[ 1  1 -1  1  3  1  2  2  2  0]
 [ 1  0 -1  1  0  0  0  0  0  0]
 [ 0  2 -1  2  4  1  2  0  0  0]]


### Question 1.3 [10 Marks]
We can now learn the parameters by maximum likelihood estimation for some of the variables. 
Let $X$ denote the observed variables, $X=[D,E,C,G,I,T,F,S,J]$, and $Z$ denote the unobserved variables, $Z=[A]$. We can denote assignments to the variables by $\alpha=\{x_n,z_n\}_{n=1}^N$ where $x_n$ are the observed assignments for the $n^\text{th}$ sample and $z_n$ are the unobserved assignments. So $x_n^{(D,d)}=1$ if in the $n^\text{th}$ sample the value of $D$ was state $d$, and $x_n^{(D,d)}=0$ otherwise. Likewise the unknown assignments for the latent variables are denoted $z_n^{(A,a)}\in{0,1}$. 

Also let us represent the parameters of a variable of the model, which represent the conditional probability for that variable w.r.t. its parents $p(\text{var} \mid \text{pa(var)})$, by $\theta^{var}$. For example, 
$$p(G=g \mid D=d,A=a,C=c)=\theta^{(G)}_{d,a,c,g}.$$

Then we can represent the joint probability in this way:
\begin{align*}
p(X,Z|\alpha,\theta)
    &=\prod_{n}\left(p(D|x_n,z_n,\theta^{(D)}) ... p(G|x_n,z_n,D,A,C,\theta^{(G)}) ...\right)\\
    &=\prod_{n}\left(\left(\prod_{d\in D}(\theta^{(D)}_d)^{x_n^{(D,d)}}\right) ... \left(\prod_{d\in D}\prod_{a\in A}\prod_{c\in C}\prod_{g\in G}(\theta^{(G)}_{d,a,c,g})^{x_n^{(D,d)}z_n^{(A,a)}x_n^{(C,c)}x_n^{(G,g)}}\right) ... \right)\\
\end{align*}

Note we have the values $x_n$ but do not have the values $z_n$.

1. Which variables can we derive the MLE solution for their parameters for (note this would mean representing their parameters in terms of $x_n$ only, without $z_n$). Explain why.

2. (M-step for easy variables)
Show that the MLE solution for $\theta^{(D)}$ is given by
$$\theta^{(D)}_d = \frac{\sum_n x_n^{(D,d)}}{\sum_{d'}\sum_n x_n^{(D,d')}}$$
and argue that we can generalise to a general MLE solution in terms of a variable $V$ and its ancestors $U_1,...,U_n$
$$\theta^{(V)}_{{u_1}...{u_n}v} = \frac{\sum_n x_n^{(U_1,{u_1})}...x_n^{(U_n,{u_n})}x_n^{(V,v)}}{\sum_{v'}\left(\sum_n x_n^{(U_1,{u_1})}...x_n^{(U_n,{u_n})}x_n^{(V,v')}\right)}$$

 (Hint: maximise the log of the joint probability, and use Lagrange Multipliers to enforce that the sum of conditionals should sum to 1).

3. Code up the general MLE solution and use it to calculate the parameters for the variables you listed in part 1. Update the pmfs of the nodes made in Q1/2 have these parameters. For the $x_n$'s we suggest you use the assignments array we make below, and fit the rest of the solution using the template we provide.

### Solution

1.3.1  
Answer are D E C T  
Since they don't have latent variables as their ancestors and they are not latent variables. 

1.3.2   
Firstly, since $\theta^{(D)}_d$ is the conditional probability of $\mathbf{D}$, we have $\sum_{d}\theta_{d}^{(D)} = 1$
Then, we will represent the log of the joint probability and use Lagrange Multipliers:

\begin{align*}
  L &= \log{p(X,Z|\alpha,\theta)} + \lambda \left(\sum_{d}\theta_{d}^{(D)}-1\right)\\
    &=\sum_{n}\left(p(D|x_n,z_n,\theta^{(D)}) ... +p(G|x_n,z_n,D,A,C,\theta^{(G)}) ...\right)\\
    &=\sum_{n}\left(\left(\sum_{d\in D}\log{(\theta^{(D)}_d)} x_n^{(D,d)}\right) ... +\left(\sum_{d\in D}\sum_{a\in A}\sum_{c\in C}\sum_{g\in G}\log{(\theta^{(G)}_{d,a,c,g})}x_n^{(D,d)}z_n^{(A,a)}x_n^{(C,c)}x_n^{(G,g)}\right) ... \right)+\lambda \left(\sum_{d}\theta_{d}^{(D)}-1\right)
\end{align*}

Then we have two partial derivatives of $L=0$:  
$$\frac{\partial L}{\partial \theta_{d}^{(D)}} = \frac{\sum_{n}x_{n}^{(D,d)}}{\theta_{d}^{(D)}} + \lambda = 0 \text{ (1), and d is specific here.}$$   
$$\frac{\partial L}{\partial \lambda} = \sum_{d}\theta_{d}^{(D)} - 1 = 0$$
Then we have: $$\sum_{d'}\sum_{n}x_{n}^{(D,d')}  = - \sum_{d'}\theta_{d'}^{(D)}\lambda \text{  , } d' \text{ is universal here.}$$  
Also we have $\sum_{d'}\theta_{d'}^{(D)} = 1$ from $\frac{\partial L}{\partial\lambda}$:
$$\sum_{d'}\sum_{n}x_{n}^{(D,d')}  = - \lambda \text{(2)}$$ 
From (1), (2):
$$\frac{\sum_{n}x_{n}^{(D,d)}}{\theta_{d}^{(D)}} = \sum_{d'}\sum_{n}x_{n}^{(D,d')}$$

Therefore, $$\theta^{(D)}_d = \frac{\sum_n x_n^{(D,d)}}{\sum_{d'}\sum_n x_n^{(D,d')}}$$

Similiarly, we assume there are a variable $V$ and its ancestors $U_1,...U_n$,  
the log of the joint probability and using Lagrange Multipliers:  

$$L = \log{p(X,Z|\alpha,\theta)} + \lambda \left(\sum_{v}\theta^{(V)}_{{u_1}...{u_n}v}-1\right)$$  
Assmue two partial derivatives of $L=0$, same as easy variables:   
$$\frac{\partial L}{\partial \theta^{(V)}_{{u_1}...{u_n}v}} =  \frac{\sum_{n}x_n^{(U_1,u_1)}...x_n^{(U_n,u_n)}x_n^{(V,v)}}{\theta^{(V)}_{{u_1}...{u_n}v}} + \lambda =0 \text{ (1), and }v \text{ is specific here.} $$    

$$\frac{\partial L}{\partial\lambda} = \sum_{v}\theta^{(V)}_{{u_1}...{u_n}v}-1=0$$

Then, we have: 
$$\sum_{v'}\sum_{n}x_n^{(U_1,u_1)}...x_n^{(U_n,u_n)}x_n^{(V,v')} = -\lambda \sum_{v'}\theta^{(V)}_{{u_1}...{u_n}v'}\text{ , and }v' \text{ is universal here.}$$
Also we have $\sum_{v'}\theta^{(V)}_{{u_1}...{u_n}v'} = 1$ from $\frac{\partial L}{\partial\lambda}$:  
$$-\lambda = \sum_{v'}\left(\sum_n x_n^{(U_1,{u_1})}...x_n^{(U_n,{u_n})}x_n^{(V,v')}\right)  \text{ (2)}$$

Therefore, from (1), (2):
$$\theta^{(V)}_{{u_1}...{u_n}v} = \frac{\sum_n x_n^{(U_1,{u_1})}...x_n^{(U_n,{u_n})}x_n^{(V,v)}}{\sum_{v'}\left(\sum_n x_n^{(U_1,{u_1})}...x_n^{(U_n,{u_n})}x_n^{(V,v')}\right)}$$

In [7]:
## Supporting Cell Do NOT change

# Intialise assignments
n, v = sample_idxes.shape
assignments = np.zeros((n, v, 5)) # 5 as that is the maximum number of states of any variable
for i in range(n):
    for j in range(v):
        assignments[i,j,sample_idxes[i,j]] = 1
assignments[:,nodes.index(A)] = np.nan # No assignments as it (A) is latent latent

In [8]:
def calculate_parameters(curr_var, assignments):
    # Calculate the shape of the pmf array for the cur_var
    pmf_shape = tuple(len(var.states) for var in curr_var.pmf.dims)
    # Initialise a pmf array of the shape calculated above with 0 values
    var_pmf = np.zeros(pmf_shape)

    # TODO: Use assignments to calculate the values for the pmf array

    curr_idx = None
    parents_idx = []
    curr_parents = curr_var.parents
    for idx,single_node in enumerate(nodes):
        if curr_var == single_node:
            curr_idx = idx
        for parent in curr_parents:
            if parent == single_node:
                parents_idx.append(idx)
                
#     print(var_pmf.shape)
#     print(curr_idx)
#     print(parents_idx)
    # i.i.d variables
    if curr_parents == []:
        count_curr = assignments[:,curr_idx,0:curr_var.num_states]
        sum_curr = sum(count_curr)
        total =sum(sum_curr)
        var_pmf = sum_curr/total
    else:
        # dependent variables
        # count
        for idx_n, single_student_data in enumerate(assignments):
            curr_assignment = single_student_data[curr_idx]
            curr_value = np.argwhere(curr_assignment==1)[0]
            single_needed = single_student_data[parents_idx]
            sample = np.argwhere(single_needed==1)

            indexing_parents = sample[:,1]            
            indexing_total = np.hstack((indexing_parents,curr_value))

            var_pmf[tuple(indexing_total)]+=1

#         print(var_pmf)
        # normalize
        parents_dims = []
        for idx in parents_idx:
            parents_dims.append(list(range(nodes[idx].num_states)))
#         print(parents_dims)
        all_products = parents_dims[0]
        for i,dims in  enumerate(parents_dims):
            if i !=0:
                all_products = itertools.product(all_products,dims)
        
        for indexing in all_products:
#             print(var_pmf[indexing])
            total = sum(var_pmf[indexing])
            var_pmf[tuple(indexing)]/=total
            
#         print(var_pmf)
        
    # assign the newly calculated pmf values to the curr_var's pmf array
    curr_var.pmf.values = var_pmf

In [9]:
## Supporting Cell Do NOT change

# list the nodes that we can derive the MLE solution for
mle_nodes = [D,E,C,T] # TODO: fill this in 
try:
    for node in mle_nodes:
        calculate_parameters(node, assignments)    
except NotImplementedError:
    print("Need to implement calculate_parameters")
if len(mle_nodes) == 0:
    print("Need to add nodes to mle_nodes")
D.pmf.values # Should be roughly [0.2,0.5,0.3]

array([0.2058, 0.4964, 0.2978])

### Question 1.4 [10 Marks]

We now turn to how to properly train our latent variable model using the Expectation Maximization algorithm. Feel free to refer to Section 9.3 and Section 9.4 of the Bishop book for a refresher of this content. 

The objective of EM is to maximise the expectation of the complete log-likelihood w.r.t. the latent variables:
\begin{align*}
E_Z[\log p(X,Z \mid \alpha,\theta)]=\sum_Zp(Z\mid X, \alpha, \theta)\log p(X,Z \mid \alpha,\theta).
\end{align*}

In the E-step we should be calculate the expected value of assignments to the latent variables with respect to the data and the current parameters. Thus as $A$ is the only latent parameter, we should be able to calculate the expected value of $z_n^{(A,a)}$, which we can denote by $E\left[z_n^{(A,a)}\right]$.

Furthermore, as the values of $z_n^{(A,a)}$ are 0 or 1, we have

\begin{align*}
E\left[z_n^{(A,a)}\right]
    &=\sum_{v\in\{0,1\}} v*p\left(z_n^{(A,a)}=v\mid x_n,\theta\right)\\
    &= p\left(z_n^{(A,a)}=1\mid x_n,\theta\right)\\
    &= \frac{p\left(x_n,z_n^{(A,a)}=1\mid \theta\right)}{p(x_n\mid \theta)}
\end{align*}


1. (M-step for hard variables)  Given such expected values, we should be able to use MLE to solve for the parameters of all other variables, and get the exact same form as the general solution given in part 2. of the last question. More specifically given latent variable $V$ with observed ancestors $U_i$ and unobserved ancestors $W_i$, where all the latent variables are independent of each other (i.e. the $W_i$ and $V$ are all independent from each other), then the parameters for $V$ can be given by 
$$\theta^{(V)}_{{u_1}...{w_1}...v} = \frac{\sum_n x_n^{(U_1,{u_1})}...E\left[z_n^{(W_1,{w_1})}\right]...E\left[z_n^{(V,v)}\right]}{\sum_{v'}\left(\sum_n x_n^{(U_1,{u_1})}...E\left[z_n^{(W_1,{w_1})}\right]...E\left[z_n^{(V,v')}\right]\right)}.$$
Show this by solving for the parameters that maximise the EM objective (and again use Lagrange Multipliers, the steps should be very similar).

2. (E-step) Note that the equation given in 1. can be implemented using the exact same function we wrote in Q1 (calculate_parameters) where the values used in assignments are the expectations $E\left[z_n^{(A,a)}\right]$. To calculate these expectations, implement the sample_ll function, which given a single instances' sample values (i.e. the value of each variable in the network) calculates the log probability of that sample under the parameters. Use our template to do this, which will work in the log domain as otherwise we would be multiplying lots of small probabilities which leads to numerical instability. The supporting code will use this function to calculate the expectations and store them in the assignments array.

### Solution
 {Type your solution to 1.4.1 and the "show" part of 1.4.2 here}

Since we have:
\begin{align*}
E\left[z_n^{(A,a)}\right]
    &=\sum_{v\in\{0,1\}} v*p\left(z_n^{(A,a)}=v\mid x_n,\theta\right)\\
    &= p\left(z_n^{(A,a)}=1\mid x_n,\theta\right)\\
    &= \frac{p\left(x_n,z_n^{(A,a)}=1\mid \theta\right)}{p(x_n\mid \theta)}
\end{align*}
Then, we will represent the log of the joint probability and use Lagrange Multipliers:
\begin{align*}
  L &= E_Z[\log p(X,Z \mid \alpha,\theta)] + \lambda \left(\sum_{v}\theta^{(V)}_{{u_1}...{w_1}...v}-1\right)\\
    &=E[\sum_{n}\left(\left(\sum_{d\in D}\log{(\theta^{(D)}_d)} x_n^{(D,d)}\right) ... +\sum_{d\in D}\sum_{a\in A}\sum_{c\in C}\sum_{g\in G}\log{(\theta^{(G)}_{d,a,c,g})}x_n^{(D,d)}z_n^{(A,a)}x_n^{(C,c)}x_n^{(G,g)}\right) ... ] + \lambda \left(\sum_{v}\theta^{(V)}_{{u_1}...{w_1}...v}-1\right)\\
\end{align*}

Because the property of expectation $E[ma+ n] = mE[a]+n$, we have: 
\begin{align*}
L =\sum_{n}\left(\left(\sum_{d\in D}\log{(\theta^{(D)}_d)} x_n^{(D,d)}\right) ... +\left(\sum_{d\in D}\sum_{a\in A}\sum_{c\in C}\sum_{g\in G}\log{(\theta^{(G)}_{d,a,c,g})}x_n^{(D,d)}E[z_n^{(A,a)}]x_n^{(C,c)}x_n^{(G,g)}\right) ... \right) + \lambda \left(\sum_{v}\theta^{(V)}_{{u_1}...{w_1}...v}-1\right)
\end{align*}
Then, we can assmue two partial derivatives of $L=0$ as 1.3 did:  
$$\frac{\partial L}{\partial \theta^{(V)}_{{u_1}...{w_1}...v}} = \frac{\sum_n x_n^{(U_1,{u_1})}...E\left[z_n^{(W_1,{w_1})}\right]...E\left[z_n^{(V,v)}\right]}{\theta^{(V)}_{{u_1}...{w_1}...v}} + \lambda = 0 \text{ (1), and }𝑣\text{ is a specific latent variable here.}$$  
$$\frac{\partial L}{\partial \lambda} = \sum_{v}\theta^{(V)}_{{u_1}...{w_1}...v}-1 = 0$$
We can still get the universal latent variable $v'$ equations by the sum of the previous specific equation:  
$$ \sum_{v'}\sum_n x_n^{(U_1,{u_1})}...E\left[z_n^{(W_1,{w_1})}\right]...E\left[z_n^{(V,v')}\right] = - \sum_{v'}\theta^{(V)}_{{u_1}...{w_1}...v'} \lambda$$
Also we have $\sum_{v'}\theta^{(V)}_{{u_1}...{w_1}...v'} = 1$ from $\frac{\partial L}{\partial\lambda}$:
$$ - \lambda = \sum_{v'}\sum_n x_n^{(U_1,{u_1})}...E\left[z_n^{(W_1,{w_1})}\right]...E\left[z_n^{(V,v')}\right] \text{ (2)}$$  
Therefore,from (1) and (2):  
$$\theta^{(V)}_{{u_1}...{w_1}...v} = \frac{\sum_n x_n^{(U_1,{u_1})}...E\left[z_n^{(W_1,{w_1})}\right]...E\left[z_n^{(V,v)}\right]}{\sum_{v'}\left(\sum_n x_n^{(U_1,{u_1})}...E\left[z_n^{(W_1,{w_1})}\right]...E\left[z_n^{(V,v')}\right]\right)}.$$

In [10]:
def sample_ll(nodes, single_sample_idxes):
    # Assume that nodes is in the topological order, and single_sample_idxes has the state
    # index of each variable (including latent variable) for a one instance.
    single_sample_idxes = np.array(single_sample_idxes).reshape(-1)
    assert len(nodes) == len(single_sample_idxes)
    ll = 0
    
    # TODO: Sum up the log probs of each 
    for idx, value in enumerate(single_sample_idxes):
        curr_node = nodes[idx]
        curr_parents = curr_node.parents
        
        if len(curr_parents)>0:
            curr_parents_idx = [nodes.index(parent) for parent in curr_parents]
            curr_parents_idx.append(idx)
#             print(curr_node)
#             print(curr_parents_idx)
#             print(curr_node.pmf.values.shape)
#             print(single_sample_idxes)
            indexes = single_sample_idxes[curr_parents_idx]
#             print(indexes)

            
            curr_log = np.log(curr_node.pmf.values[tuple(indexes)])
        else:
            curr_log = np.log(curr_node.pmf.values[value])
        ll+=curr_log

    return ll

In [11]:
## Supporting Cell Do NOT change
try:
    # Initialise the assignments to 0
    intel_assignments = np.zeros((len(sample_idxes), 5))
    for i in range(len(sample_idxes)):
        instance_probs = []
        for state in range(5):
            data = np.array(sample_idxes[i])
            data[2] = state # Set Intel's value to state
            ll = sample_ll(nodes, data)
            instance_probs.append(ll)
        instance_probs = np.exp(np.array(instance_probs))
        intel_assignments[i] = instance_probs/instance_probs.sum() # normalise ourselves
    print(intel_assignments[0]) # Should be approx [0.01,0.02,0.03,0.14,0.80]
except NotImplementedError:
    print("Need to implement sample_ll")

[0.01210654 0.02421308 0.02905569 0.13559322 0.79903148]


### Running the EM algorithm
Now we can run our EM algorithm to completion.

In [12]:
## Supporting Cell Do NOT change
try:
    print("Initial A marginals: ", A.pmf.values)
    old_norm_diff = 1
    for it in range(100):
        # E step
        apt_assignments = np.zeros((len(sample_idxes), 5))
        for i in range(len(sample_idxes)):
            instance_probs = []
            for state in range(5):
                data = np.array(sample_idxes[i])
                data[2] = state
                ll = sample_ll(nodes, data)
                instance_probs.append(ll)
            instance_probs = np.exp(np.array(instance_probs))
            apt_assignments[i] = instance_probs/instance_probs.sum()
        # Set assignments made in E-step
        assignments[:,2,:] = apt_assignments
        # M step
        old_A_pmfs = np.array(A.pmf.values)
        for curr_var in nodes:
            if curr_var in [S, F]: # Do not override the parameters we loaded in
                continue
            ind = [nodes.index(var) for var in curr_var.pmf.dims] # [0, 1, 2, 4]
            pmf_shape = tuple(len(var.states) for var in curr_var.pmf.dims) # (3,3,5,5)
            var_pmf = np.zeros(pmf_shape)
            for values in itertools.product(*(range(len(var.states)) for var in curr_var.pmf.dims)):
                var_pmf[values] = assignments[:,ind,values].prod(axis=1).sum()
            var_pmf /= var_pmf.sum(axis=-1, keepdims=True)
            curr_var.pmf.values = var_pmf
        # Check for convergence
        norm_diff = np.linalg.norm(old_A_pmfs-A.pmf.values)
        if (old_norm_diff-norm_diff)/old_norm_diff < 0.005:
            print("Converged")
            break
        old_norm_diff = norm_diff
        print("Iteration {}, A marginals: {}, Norm Diff {:.5f}".format(it, A.pmf.values, norm_diff)) 
        # Should get to roughly [0.05, 0.2, 0.35, 0.3, 0.1], something like [0.08, 0.18, 0.36, 0.27, 0.11]
except NotImplementedError:
    print("Need to implement sample_ll!")

Initial A marginals:  [0.2 0.2 0.2 0.2 0.2]
Iteration 0, A marginals: [0.18114851 0.19602761 0.21529265 0.21455572 0.19297552], Norm Diff 0.02943
Iteration 1, A marginals: [0.16991324 0.19261293 0.22531907 0.2245983  0.18755646], Norm Diff 0.01920
Iteration 2, A marginals: [0.16142276 0.19012827 0.2343292  0.23291535 0.18120442], Norm Diff 0.01640
Iteration 3, A marginals: [0.15401524 0.18823729 0.24332191 0.24034539 0.17408017], Norm Diff 0.01566
Iteration 4, A marginals: [0.14727167 0.18668574 0.2522383  0.24700604 0.16679825], Norm Diff 0.01499
Iteration 5, A marginals: [0.14111844 0.18535221 0.26084245 0.25287024 0.15981666], Norm Diff 0.01403
Iteration 6, A marginals: [0.13553879 0.18418671 0.26895989 0.25793621 0.15337839], Norm Diff 0.01286
Iteration 7, A marginals: [0.13050752 0.18316648 0.27650328 0.26224729 0.14757543], Norm Diff 0.01164
Iteration 8, A marginals: [0.12598723 0.18227507 0.28344895 0.26587547 0.14241328], Norm Diff 0.01045
Iteration 9, A marginals: [0.12193482 

## BN Part 3: Inference with the learned model

If you have not finished Part 2 and don't have a trained model, load in the following parameters here.

In [13]:
import pickle
part2complete = False # Change this if you have not completed part 2!!!
if not part2complete:
    with open(os.path.join("data","question_1","saved_pmf_vals.pkl"), 'rb') as fp:
        saved_pmf_vals = pickle.load(fp)

    for i, var in enumerate(nodes):
        var.pmf.values = saved_pmf_vals[i]

### Question 1.5 [10 Marks]
1. Calculate the following marginals/conditionals using the `sample_idxes` data: $p(J)$, $p(G)$, $p(I)$, $p(J|G)$, $p(J|C)$, $p(J|E)$.

2. Implement a function that marginalises over variables. That is, given variables to keep, it sums over all other variables. For example, given 
`marginaliseConditionals(nodes, [C])` it will sum over all other variables except for $C$. Thus in $C$ it would leave $p(C)$, in $I$ it would leave $p(I|C)$, in $A$ it would leave $p(A)$ as it doesn't have $C$ as an acestor, and in $J$ it would leave $p(J|C)$. Use this to calculate $p(J|C)$, $p(J|A)$, $p(J|E)$ and $p(J|E,A)$. Check your code by comparing to the calculations done above (they should only have slightly differing values). Furthermore calculate $p(J|A)$, which we couldn't do with the sample_idxes.

 (Hint: use the `multiply_pmf` function provided).

3. Calculate $p(E|J)$ (`marginaliseConditionals` should be useful here).

In [14]:
# 1.5.1: Calculate the marginals/conditionals from the data and print them out here
# calculate p(J)
jobs = np.zeros((2))
for single in sample_idxes:
    if single[-1]==0:
        jobs[0]+=1
    else:
        jobs[1]+=1
jobs/=sum(jobs)
print("p(J)")
print(jobs)

#calculate p(G)
grades = np.zeros((5))
for single in sample_idxes:
    curr_idx = single[4]
    grades[curr_idx]+=1
grades/=sum(grades)
print("p(G)")
print(grades)

#calculate p(I)
interviews = np.zeros((3))
for single in sample_idxes:
    curr_idx = single[5]
    interviews[curr_idx]+=1
interviews/=sum(interviews)
print("p(I)")
print(interviews)

#calculate p(J|G)
jobs_given_grades = np.zeros((5,2))
for single in sample_idxes:
    curr_idx = single[-1]
    grade_idx = single[4]
    jobs_given_grades[grade_idx][curr_idx]+=1
for i in range(5):
    jobs_given_grades[i]/=sum(jobs_given_grades[i])
print("p(J|G)")
print(jobs_given_grades)

#calculate p(J|C)
jobs_given_confidence = np.zeros((3,2))
for single in sample_idxes:
    curr_idx = single[-1]
    confidence_idx = single[3]
    jobs_given_confidence[confidence_idx][curr_idx]+=1
for i in range(3):
    jobs_given_confidence[i]/=sum(jobs_given_confidence[i])
print("p(J|C)")
print(jobs_given_confidence)

#calculate p(J|E)
jobs_given_effort = np.zeros((3,2))
for single in sample_idxes:
    curr_idx = single[-1]
    effort_idx = single[1]
    jobs_given_effort[effort_idx][curr_idx]+=1
for i in range(3):
    jobs_given_effort[i]/=sum(jobs_given_effort[i])
print("p(J|E)")
print(jobs_given_effort)

p(J)
[0.8736 0.1264]
p(G)
[0.2044 0.1786 0.1968 0.185  0.2352]
p(I)
[0.4616 0.4314 0.107 ]
p(J|G)
[[0.95303327 0.04696673]
 [0.95968645 0.04031355]
 [0.94817073 0.05182927]
 [0.88432432 0.11567568]
 [0.66836735 0.33163265]]
p(J|C)
[[0.88341232 0.11658768]
 [0.87316239 0.12683761]
 [0.86470588 0.13529412]]
p(J|E)
[[0.92983342 0.07016658]
 [0.87142857 0.12857143]
 [0.77242682 0.22757318]]


In [15]:
## Supporting Cell Do NOT change
def multiply_pmfs(pmf1,pmf2,var):
    # Implements marginalising pmf2 over var using pmf1. pmf1 is not modified, but pmf2 is.
    # Assumes the following dimensions
    # pmf1.dims : (a1,....,an,var)
    # pmf2.dims : (b1,.....,bm,var2) where var (and maybe some of a1,...,an) is contained in b1,...,bm

    # Make a copy
    pmf1 = PMF(list(pmf1.dims), np.array(pmf1.values))
    assert var == pmf1.dims[-1], (var.name, vars2names(pmf1.dims))
    assert var in pmf2.dims, (var.name, vars2names(pmf2.dims))
    a_is = pmf1.dims[:-1]
    b_is = pmf2.dims[:-1]
    assert var in b_is, (var.name,vars2names(b_is))
    b_is.remove(var)

    intersect = set(a_is) & set(b_is)

    i = pmf2.dims.index(var)
    pmf2.values = np.moveaxis(pmf2.values, i, -2)
    pmf2.dims = pmf2.dims[:i] + pmf2.dims[i+1:] # Remove var
    # Now pmf2: ({b1,...,bm}\var,var,var2)
    pmf1.values = np.expand_dims(pmf1.values,axis=-2)
    # Now pmf2: ({a1,...,bn},1,var)
    # print("Intersect", vars2names(intersect))
    for var_i in intersect:
        i = pmf1.dims.index(var_i)
        pmf1.values = np.moveaxis(pmf1.values, i, -3)
        pmf1.dims = pmf1.dims[:i] + pmf1.dims[i+1:-1] + [var_i] + pmf1.dims[-1:]
        i = pmf2.dims.index(var_i)
        pmf2.values = np.moveaxis(pmf2.values, i, -3)
        # print(pmf2.dims[:i] , pmf2.dims[i+1:-2] , [var_i] , pmf2.dims[-2:])
        pmf2.dims = pmf2.dims[:i] + pmf2.dims[i+1:-1] + [var_i] + pmf2.dims[-1:]
    # Now pmf2: ({b1,...,bm}\{var}\cup {intersect},{intersect},var,var2)
    # Now pmf1: ({11,...,an}\{intersect},{intersect},1,var2)
    # print('1:', pmf1, pmf2)
    pmf2_other = set(b_is) - intersect
    pmf1_other = set(a_is) - intersect
    for _ in range(len(pmf2_other)):
        pmf1.values = np.expand_dims(pmf1.values,axis=len(pmf1_other))
    pmf2.dims = pmf1.dims[:len(pmf1_other)] + pmf2.dims
    # print('2:', pmf1, pmf2)
    # Now pmf2: ({b1,...,bm}\{var}\cup {intersect},{intersect},var,var2)
    # Now pmf1: ({11,...,an}\{intersect},{1,...,1},{intersect},1,var2)
    # print("Shapes", pmf1.values.shape, pmf2.values.shape)
    pmf2.values = pmf1.values @ pmf2.values
    # Now ({11,...,an}\{intersect},{b1,...,bm}\{var}\cup {intersect},{intersect},1,var2)
    pmf2.values = np.squeeze(pmf2.values, axis=-2)
    # Now ({11,...,an}\{intersect},{b1,...,bm}\{var}\cup {intersect},{intersect},var2)

In [16]:
def marginaliseConditionals(top_order, rem_vars):
    # rem_vars are the vars to not marginalise over
    for var in top_order:
        # Make a copy
        new_pmf = PMF(list(var.pmf.dims), np.array(var.pmf.values))

        # TODO: Implement marginalising over all variables in var.pmf.dims apart from any vars in rem_var
        # Hint: use multiply_pmfs
        # intersections/remaining parents are not empty
        
#         print("ITSSSSSSSS",var)
        if len(new_pmf.dims)==1 and new_pmf.dims[0] in [D,E,A,C]:
            var.new_pmf = new_pmf
            continue
#         if var not in rem_vars:
#             rem_vars.append(var)
#         print(rem_vars)


#         print(len(set(new_pmf.dims).difference(set(rem_vars))))
        while len(set(new_pmf.dims).difference(set(rem_vars))) != 0:
            idx=0
#             print(new_pmf.dims[idx] in rem_vars or var==new_pmf.dims[idx])
            while new_pmf.dims[idx] in rem_vars or var==new_pmf.dims[idx]:
#                 print(new_pmf.dims[idx] in rem_vars or var==new_pmf.dims[idx])
                if idx == len(new_pmf.dims)-1:
                    idx = -114514
                    break
                else:
                    idx+=1
            if idx == -114514:
                break
            remove_curr = new_pmf.dims[idx]
#             print(var,remove_curr)
#             print(new_pmf.dims)
#             print()
            multiply_pmfs(remove_curr.pmf,new_pmf,remove_curr)
        
        var.new_pmf = new_pmf
#         print(var,new_pmf)

In [17]:
## Supporting Cell Do NOT change

# 1.5.2: Use marginaliseConditionals to calculate the marginals/conditionals from the trained BN and print them out here
# Check against prev values calculated from the data (if possible)
try:
    for var in [J, G, I]:
        marginaliseConditionals(nodes, [var])
        print("Marginals for {}".format(var.name))
        print(var.new_pmf.values)
        print()
    for var1, var2 in [(J, G), (J, C), (J, E)]:
        marginaliseConditionals(nodes, [var2])
        print("Marginals for {} given {}".format(var1.name, var2.name))
        print(var1.new_pmf.values)
        print()
    for var1, var2 in [(J, C), (J, A), (J, E)]:
        marginaliseConditionals(nodes, [var2])
        print("Marginals for {} given {}".format(var1.name, var2.name))
        print(var1.new_pmf.values)
        print()
    marginaliseConditionals(nodes, [E, A])
    print("Marginals for {} given {}, {}".format(J.name, E.name, A.name))
    print("Dims: ", vars2names(J.new_pmf.dims))
    print(J.new_pmf.values)
except NotImplementedError:
    print("Need to implement marginaliseConditionals!")

Marginals for Job
[0.88233116 0.11766884]

Marginals for Grade
[0.20291146 0.17875817 0.19789516 0.18499422 0.23544098]

Marginals for Interview
[0.46217175 0.430716   0.10711225]

Marginals for Job given Grade
[[0.95010592 0.04989408]
 [0.95986037 0.04013963]
 [0.94654524 0.05345476]
 [0.88732419 0.11267581]
 [0.70715949 0.29284051]]

Marginals for Job given Confidence
[[0.89808225 0.10191775]
 [0.88363667 0.11636333]
 [0.86229587 0.13770413]]

Marginals for Job given Effort
[[0.93234492 0.06765508]
 [0.8774997  0.1225003 ]
 [0.77154083 0.22845917]]

Marginals for Job given Confidence
[[0.89808225 0.10191775]
 [0.88363667 0.11636333]
 [0.86229587 0.13770413]]

Marginals for Job given Aptitute
[[0.92812449 0.07187551]
 [0.91948149 0.08051851]
 [0.88735288 0.11264712]
 [0.85253003 0.14746997]
 [0.81464202 0.18535798]]

Marginals for Job given Effort
[[0.93234492 0.06765508]
 [0.8774997  0.1225003 ]
 [0.77154083 0.22845917]]

Marginals for Job given Effort, Aptitute
Dims:  ['Aptitute', '

In [18]:
# 1.5.1: Calculate p(E|J) here. Should use marginaliseConditionals
# We intend to use Bayesian rules to derive p(E|J) = p(J|E)p(E) / p(J)
ans = np.zeros((J.num_states,E.num_states))

marginaliseConditionals(nodes,[E])
p_J_given_E = J.new_pmf.values
p_E = E.new_pmf.values

marginaliseConditionals(nodes, [J])
p_J = J.new_pmf.values

for i in range(J.num_states):
    for j in range(E.num_states):
        ans[i][j] = p_J_given_E[j,i]*p_E[j]/p_J[i]

ans = ans/ np.sum(ans,axis=1,keepdims=True)
print("Marginals for {}".format(E.name))
print(ans)
print()

Marginals for Effort
[[0.42130521 0.39231851 0.18637627]
 [0.21755015 0.38973346 0.39271639]]

